# Midterm

The midterm project will consist of a comparison between several CNN architectures for prostate segmentation. The goal is both to create a high-performing algorithm for the target task, as well as to analyze performance across several different architecture permutations. In total, three different network designs will be tested. As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for all three models
* final compiled `*.csv` file with performance statistics across the different architectures
* final 1-page write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [ ]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [ ]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [ ]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [ ]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets, custom

# Data

As in the tutorial, data for this assignment will consist of prostate MRI exams. In prior work, an algorithm was created to separate out different MRI sequences. In this current assignment, only T2-weighted images (isolated using the prior algorithm) will be used for segmentation. In prostate imaging, the T2-weighted sequence captures the greatest amount of anatomic detail and is thus ideal for delineation of prostate gland structures.

The following lines of code will download the dataset (if not already present). Since each algorithm below requires a different dataset, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

In [ ]:
# --- Download dataset
datasets.download(name='mr/prostatex-seg')

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. 2D U-Net (single step)

In this algorithm a standard 2D U-Net architecture will be used to perform prostate segmentation. This network is **identical** to the week 5 assignment. The algorithm input will include an entire full field-of-view `256 x 256` resolution 2D slice from a T2 weighted MR image. Key customizations to the standard U-Net architecture that should be implemented (as in the week 5 assignment) include:

* same padding (vs. valid padding)
* strided convolutions (vs. max-pooling)

### Create generators and inputs

In [ ]:
# --- Original 256 x 256 (one-step)
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-256')
inputs = client.get_inputs(Input)

### Define model

In [ ]:
# --- Define model

# --- Create logits
logits = {}
logits['zones'] = layers.Conv3D(filters=3, name='zones', ...)(...)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile and train model

In [ ]:
# --- Compile model
model.compile(...)

# --- Train the model
model.fit(...)

## 2. 2D U-Net (multiple step)

In this algorithm, the output of the first 2D U-Net will be used to generated a cropped `128 x 128` resolution 2D slice centered around the prostate gland. This method effectively focuses the algorithm field-of-view to the area of interest and helps improve on inherent class imbalance associated with this task.

### Create generators and inputs

In [ ]:
# --- Cropped 128 x 128 (multiple step)
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-crp')
inputs = client.get_inputs(Input)

### Define model

In [ ]:
# --- Define model

# --- Create logits
logits = {}
logits['zones'] = layers.Conv3D(filters=3, name='zones', ...)(...)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile and train model

In [ ]:
# --- Compile model
model.compile(...)

# --- Train the model
model.fit(...)

## 3. Custom architecture (multiple step)

Finally, using all customizations described in class, find a top-performing model that yields some incremental benefit over the two baseline models above. A multi-step approach (using the cropped `128 x 128` inputs) will tend to yield improved results. Additional modifications that be used include (but are not limited to):

* hybrid 3D/2D network
* residual connections
* added convolutions between contracting and expanding layers 

In [ ]:
# --- Select shape
configs = {'specs': {'xs': {'dat': {'shape': [3, 128, 128, 1]}}}}

# --- Cropped 128 x 128 (multiple step)
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', keyword='seg-crp', configs=configs)
inputs = client.get_inputs(Input)

### Define model

In [ ]:
# --- Define model

# --- Create logits
logits = {}
logits['zones'] = layers.Conv3D(filters=3, name='zones', ...)(...)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile and train model

In [ ]:
# --- Compile model
model.compile(...)

# --- Train the model
model.fit(...)

# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* Dice score, mean
* Dice score, median
* Dice score, 25th percentile
* Dice score, 75th percentile

The Dice score values should be calculated both for peripheral and transitional zone (class 1 and 2); the Dice score for background does not need to be evaluated. As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

1. 2D U-Net, single step (full 256 x 256)

* peripheral zone: mean Dice score > 0.75
* transitional zone: mean Dice score > 0.55

2. 2D U-Net, multiple step (cropped)

* peripheral zone: mean Dice score > 0.80
* transitional zone: mean Dice score > 0.60

3. Custom architecture

* peripeheral zone: mean Dice score > 0.85
* transitional zone: mean Dice score > 0.65

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRANSITIONAL ZONE                       PERIPHERAL ZONE
          mean | median | 25th-tile | 75th-tile | mean | median | 25th-tile | 75th-tile
model 1
model 2
model 3
```

As above, tables for both training and validation should be provided.

In [ ]:
# --- Create *.csv
                              
# --- Serialize *.csv
fname = '{}/models/organ_segmentation/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
f.to_csv(path)

# Summary

In addition to algorithm training as above, a 1-2 page write-up is required for this project. The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? Was segmentation performance within the peripheral and transitional zones affected equally in all models? Why or why not? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.